In [1]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm

In [2]:
def splitMatrix(matrix):
    diagonalMatrix = np.diag(np.log(np.diag(matrix)))
    strictlyLowerTriangularMatrix = np.tril(matrix, k = -1)
    return diagonalMatrix, strictlyLowerTriangularMatrix

In [3]:
def distanceMatrices(matrix1, matrix2):
    chol1 = np.linalg.cholesky(matrix1)
    chol2 = np.linalg.cholesky(matrix2)
    chol1D, chol1L = splitMatrix(chol1)
    chol2D, chol2L = splitMatrix(chol2)
    distanceL = np.square(np.linalg.norm(chol1L - chol2L, 'fro'))
    distanceD = np.square(np.linalg.norm(chol1D - chol2D, 'fro'))
    distance = np.sqrt(distanceL + distanceD)
    return distance

In [4]:
def manifoldKernel(X, Y, gamma):
    numberMatricesX = X.shape[0]
    numberMatricesY = Y.shape[0]
    X = X.reshape(-1, X.shape[1], X.shape[1])
    Y = Y.reshape(-1, Y.shape[1], Y.shape[1])
    pairwiseDistances = np.zeros((numberMatricesX, numberMatricesY))
    for i in range(numberMatricesX):
        for j in range(numberMatricesY):
            distance = distanceMatrices(X[i], Y[j])
            pairwiseDistances[i, j] = np.exp(-gamma * distance ** 2)

    return pairwiseDistances

In [5]:
"""
Verify manifold SVM for UCD-MyoVerse-Hand-0 Subject 0.
Use first 18 trials of each gesture for training and last 18 trials for testing.
"""

covarianceMatrices = np.load("UCDMyoVerseSubject0Data.npy")
Labels = np.load("UCDMyoVerseSubject0Labels.npy")
print(covarianceMatrices.shape)
print(Labels.shape)

numberGestures = 10
trialsPerGesture = 36
numberChannels = 12

Indices =  {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}
for i in range(len(Labels)):
    Indices[Labels[i]].append(i)

covarianceMatrixByLabels = np.zeros((numberGestures, trialsPerGesture, numberChannels, numberChannels))
for i in range(numberGestures):
    for j in range(trialsPerGesture):
        covarianceMatrixByLabels[i, j] = covarianceMatrices[Indices[i][j]]

trainFeatures = np.zeros((numberGestures * trialsPerGesture // 2, numberChannels, numberChannels))
trainLabels = np.zeros((numberGestures * trialsPerGesture // 2))
count = 0
for i in range(numberGestures):
    trainFeatures[count:count + 18] = covarianceMatrixByLabels[i, :18]
    trainLabels[count:count + 18] = [i] * 18
    count += 18

testFeatures = np.zeros((numberGestures * trialsPerGesture // 2, numberChannels, numberChannels))
testLabels = np.zeros((numberGestures * trialsPerGesture // 2))
count = 0
for i in range(numberGestures):
    testFeatures[count:count + 18] = covarianceMatrixByLabels[i, 18:]
    testLabels[count:count + 18] = [i] * 18
    count += 18

prediction = np.zeros((numberGestures * trialsPerGesture // 2))

clf = svm.SVC(kernel = 'precomputed')

kernelMatrix = manifoldKernel(trainFeatures, trainFeatures, 1)

clf.fit(kernelMatrix, trainLabels)

kernelTest = manifoldKernel(testFeatures, trainFeatures, 1)
prediction = clf.predict(kernelTest)

print("Mean accuracy is: ", np.mean(prediction == testLabels))


(360, 12, 12)
(360,)
Mean accuracy is:  0.9888888888888889


In [6]:
"""
Verify manifold SVM for Ninapro DB2-Exercise-1 Subject 0.
Use repititions [0, 2, 3, 5] for training and repititions [1, 4] for testing.
"""

covarianceMatrices = np.load("ninaproSubject0Data.npy")
Labels = np.load("ninaproSubject0Labels.npy") - 1
print(covarianceMatrices.shape)
print(Labels.shape)

numberGestures = 17
numberChannels = 12
numberRepeat = 6

trainFeatures = np.zeros((4 * numberGestures, numberChannels, numberChannels))
trainLabels = np.zeros((4 * numberGestures))

testFeatures = np.zeros((2 * numberGestures, numberChannels, numberChannels))
testLabels = np.zeros((2 * numberGestures))

repeatTrain = [0, 2, 3, 5]
repeatTest = [1, 4]

index = 0
for gesture in range(numberGestures):
    for repeat in repeatTrain:
        trainFeatures[index] = covarianceMatrices[gesture * numberRepeat + repeat]
        trainLabels[index] = gesture
        index += 1

index = 0
for gesture in range(numberGestures):
    for repeat in repeatTest:
        testFeatures[index] = covarianceMatrices[gesture * numberRepeat + repeat]
        testLabels[index] = gesture
        index += 1

prediction = np.zeros((2 * numberGestures))
    
clf = svm.SVC(kernel = 'precomputed')

kernelMatrix = manifoldKernel(trainFeatures, trainFeatures, 8)

clf.fit(kernelMatrix, trainLabels)

kernelTest = manifoldKernel(testFeatures, trainFeatures, 8)
prediction = clf.predict(kernelTest)

print("Mean accuracy is: ", np.mean(prediction == testLabels))

(102, 12, 12)
(102,)
Mean accuracy is:  1.0
